In [1]:
from dotenv import dotenv_values
from requests import get
from requests.models import HTTPError
from bs4 import BeautifulSoup

import pandas as pd
import random

In [2]:
CONFIG = dotenv_values('../.env')

In [3]:
def requisicao_api_alura():
    try:
        response = get('https://cursos.alura.com.br/api/categorias')
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'Um erro HTTP ocorreu: {http_err}')
    except Exception as err:
        print(f'Um erro ocorreu: {err}')
    else:
        return response.json()

In [ ]:
dados = requisicao_api_alura()
urls_categorias = {}
url = 'https://cursos.alura.com.br/forum/categoria-'
for item in dados:
    urls_categorias[item['nome']] = f'{url}{item["slug"]}/todos'
urls_categorias

In [4]:
def requisicao(url):
    cookies = {'caelum.login.token':CONFIG['COOKIE']}
    try:
        response = get(url, cookies=cookies)
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'Um erro HTTP ocorreu: {http_err}')
    except Exception as err:
        print(f'Um erro ocorreu: {err}')
    else:
        soup = BeautifulSoup(response.text,'html.parser')
        return soup

In [ ]:
paginas_categorias = {}
for chave, valor in urls_categorias.items():
    soup = requisicao(valor)
    ultima_pagina = int(soup.find('nav', {'class': 'busca-paginacao-links'}).get_text().split()[-1])
    paginas_categorias[chave] = random.sample(range(1, ultima_pagina+1), 250)
paginas_categorias

In [ ]:
def dataframe_links_categoria(categoria):
    titulos = []
    links = []
    for pagina in paginas_categorias[categoria]:
        soup  = requisicao(f'{urls_categorias[categoria]}/{pagina}')
        itens  = soup.find_all('a', {'class': 'forumList-item-subject-info-title-link'})
        for item in itens:
            titulos.append(item.text.strip().split('\n')[0])
            links.append(f'https://cursos.alura.com.br{item["href"]}')

    dados = pd.DataFrame({'titulo': titulos, 'links': links})
    dados.to_csv(f'../dados/{categoria}.csv')

In [ ]:
dataframe_links_categoria('Data Science')

In [ ]:
dataframe_links_categoria('Programação')

In [ ]:
dataframe_links_categoria('Inovação & Gestão')

In [ ]:
dataframe_links_categoria('Front-end')

In [ ]:
dataframe_links_categoria('DevOps')

In [ ]:
dataframe_links_categoria('UX & Design')

In [ ]:
dataframe_links_categoria('Mobile')

In [ ]:
data_science = pd.read_csv('../dados/Data Science.csv')

In [ ]:
textos = []
i = 0
for url in data_science['links']:
    i += 1
    print(i)
    soup = requisicao(url)
    textos.append(soup.find('div', {'class': 'topic-post-question-content'}).find('section', {'class': 'topic-post-content'}).text.strip())
data_science['texto'] = textos
data_science.to_csv('../dados/Data Science com textos.csv')

In [ ]:
programacao = pd.read_csv('../dados/Programação.csv')

In [ ]:
textos = []
for url in programacao['links']:
    soup = requisicao(url)
    textos.append(soup.find('div', {'class': 'topic-post-question-content'}).find('section', {'class': 'topic-post-content'}).text.strip())
programacao['texto'] = textos
programacao.to_csv('../dados/Programação com textos.csv')

In [ ]:
devops = pd.read_csv('../dados/DevOps.csv')

In [ ]:
textos = []
for url in devops['links']:
    soup = requisicao(url)
    textos.append(soup.find('div', {'class': 'topic-post-question-content'}).find('section', {'class': 'topic-post-content'}).text.strip())
devops['texto'] = textos
devops.to_csv('../dados/DevOps com textos.csv')

In [ ]:
front_end = pd.read_csv('../dados/Front-end.csv')

In [ ]:
textos = []
for url in front_end['links']:
    soup = requisicao(url)
    textos.append(soup.find('div', {'class': 'topic-post-question-content'}).find('section', {'class': 'topic-post-content'}).text.strip())
front_end['texto'] = textos
front_end.to_csv('../dados/Front-end com textos.csv')

In [ ]:
mobile = pd.read_csv('../dados/Mobile.csv')

In [ ]:
textos = []
for url in mobile['links']:
    soup = requisicao(url)
    textos.append(soup.find('div', {'class': 'topic-post-question-content'}).find('section', {'class': 'topic-post-content'}).text.strip())
mobile['texto'] = textos
mobile.to_csv('../dados/Mobile com textos.csv')

In [ ]:
textos

In [ ]:
mobile[mobile['links'] == 'https://cursos.alura.com.br/forum/topico-realizar-o-curso-atraves-da-ide-do-android-studio-18311']

In [ ]:
mobile.drop(3720, axis = 0, inplace = True)

In [ ]:
for url in mobile[3720:]['links']:
    soup = requisicao(url)
    textos.append(soup.find('div', {'class': 'topic-post-question-content'}).find('section', {'class': 'topic-post-content'}).text.strip())
mobile['texto'] = textos
mobile.to_csv('../dados/Mobile com textos.csv')

In [ ]:
ux_design = pd.read_csv('../dados/UX & Design.csv')

In [ ]:
textos = []
for url in ux_design['links']:
    soup = requisicao(url)
    textos.append(soup.find('div', {'class': 'topic-post-question-content'}).find('section', {'class': 'topic-post-content'}).text.strip())
ux_design['texto'] = textos
ux_design.to_csv('../dados/UX & Design com textos.csv')

In [ ]:
inovacao = pd.read_csv('../dados/Inovação & Gestão.csv')

In [ ]:
textos = []
for url in inovacao['links']:
    soup = requisicao(url)
    textos.append(soup.find('div', {'class': 'topic-post-question-content'}).find('section', {'class': 'topic-post-content'}).text.strip())
inovacao['texto'] = textos
inovacao.to_csv('../dados/Inovação e Gestão com textos.csv')

Coletando a informação de tópicos Off-Topic sem resposta

In [11]:
url = "https://cursos.alura.com.br/forum/categoria-offtopic/sem-resposta"
soup = requisicao(url)
ultima_pagina = int(soup.find('nav', {'class': 'busca-paginacao-links'}).get_text().split()[-1])
titulos = []
links = []
for pagina in range(1, ultima_pagina+1):
    soup  = requisicao(f'{url}/{pagina}')
    itens  = soup.find_all('a', {'class': 'forumList-item-subject-info-title-link'})
    for item in itens:
        titulos.append(item.text.strip().split('\n')[0])
        links.append(f'https://cursos.alura.com.br{item["href"]}')

dados = pd.DataFrame({'titulo': titulos, 'links': links})
dados.to_csv(f'../dados/Off-topic.csv')

In [12]:
off_topic = pd.read_csv('../dados/Off-topic.csv')

In [14]:
textos = []
for url in off_topic['links']:
    soup = requisicao(url)
    textos.append(soup.find('div', {'class': 'topic-post-question-content'}).find('section', {'class': 'topic-post-content'}).text.strip())
off_topic['texto'] = textos
off_topic.to_csv('../dados/Off-topic com textos.csv', index = False)